In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset
import torchvision
from torchvision import models
from tqdm import tqdm
import pickle

/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
import os.path as osp
import numpy as np
from torchvision.datasets import ImageFolder


class ImageNette(ImageFolder):
    test_frac = 0.2

    def __init__(self, train=True, transform=None, target_transform=None, root="./data/"):
        root = osp.join(root, 'imagenette-160')
        if not osp.exists(root):
            raise ValueError('Dataset not found at {}. Please download it from {}.'.format(
                root, 'https://s3.amazonaws.com/fast-ai-imageclas/imagenette-160.tgz'
            ))

        # Initialize ImageFolder
        super().__init__(root=osp.join(root, 'train'), transform=transform,
                         target_transform=target_transform)
        self.root = root

        self.partition_to_idxs = self.get_partition_to_idxs()
        self.pruned_idxs = self.partition_to_idxs['train' if train else 'test']

        # Prune (self.imgs, self.samples to only include examples from the required train/test partition
        self.samples = [self.samples[i] for i in self.pruned_idxs]
        self.imgs = self.samples

        print('=> done loading {} ({}) with {} examples'.format(self.__class__.__name__, 'train' if train else 'test',
                                                                len(self.samples)))

    def get_partition_to_idxs(self):
        partition_to_idxs = {
            'train': [],
            'test': []
        }

        # Note: we perform a 80-20 split of imagenet training
        # While this is not necessary, this is to simply to keep it consistent with the paper
        prev_state = np.random.get_state()
        np.random.seed(123)

        idxs = np.arange(len(self.samples))
        n_test = int(self.test_frac * len(idxs))
        test_idxs = np.random.choice(idxs, replace=False, size=n_test).tolist()
        train_idxs = list(set(idxs) - set(test_idxs))

        partition_to_idxs['train'] = train_idxs
        partition_to_idxs['test'] = test_idxs

        np.random.set_state(prev_state)

        return partition_to_idxs
        
root = './../../data/'
transform_train = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
transform_test = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
train_set = ImageNette(train=True, transform=transform_train, root = root)
test_set = ImageNette(train=False, transform=transform_test, root = root)
batch_size = 3000 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
data_iter = iter(train_loader)
X_sub, _ = next(data_iter)
X_sub = X_sub.numpy()
print(X_sub)
print(len(X_sub))

=> done loading ImageNette (train) with 10316 examples
=> done loading ImageNette (test) with 2578 examples
[[[[ 1.3241715   1.3241715   1.3241715  ...  0.00556549 -0.04580877
    -0.06293353]
   [ 1.3241715   1.3241715   1.3241715  ...  0.07406463  0.00556549
    -0.01155927]
   [ 1.3241715   1.3241715   1.3241715  ...  0.1939379   0.09118938
     0.07406463]
   ...
   [ 1.6666666   1.7180408   1.8550389  ...  0.7761795   0.7761795
     0.7761795 ]
   [ 1.5810429   1.6837914   1.8892884  ...  0.79330426  0.7761795
     0.7761795 ]
   [ 1.5467933   1.6666666   1.923538   ...  0.79330426  0.7761795
     0.7761795 ]]

  [[ 1.4656863   1.4656863   1.4656863  ...  0.2752102   0.2401962
     0.2226892 ]
   [ 1.4656863   1.4656863   1.4656863  ...  0.3277312   0.29271722
     0.2752102 ]
   [ 1.4656863   1.4656863   1.4656863  ...  0.4327732   0.3802522
     0.3627452 ]
   ...
   [ 1.8158263   1.8683473   2.0084033  ...  0.9229692   0.9229692
     0.9229692 ]
   [ 1.7282913   1.8333333   2.0

In [3]:
import os.path as osp
import numpy as np
from torchvision.datasets import ImageFolder

class ImageWoof(ImageFolder):
    test_frac = 0.2 

    def __init__(self, train=True, transform=None, target_transform=None, root="./data"):
        root = osp.join(root, 'imagewoof2-160') 
        if not osp.exists(root):
            raise ValueError('Dataset not found at {}. Please download it from {}.'.format(
                root, '' 
            ))
        super().__init__(root=osp.join(root, 'train'), transform=transform,
                         target_transform=target_transform)
        self.root = root

        self.partition_to_idxs = self.get_partition_to_idxs()
        self.pruned_idxs = self.partition_to_idxs['train' if train else 'test']

        self.samples = [self.samples[i] for i in self.pruned_idxs]
        self.imgs = self.samples

        print('=> done loading {} ({}) with {} examples'.format(self.__class__.__name__, 'train' if train else 'test',
                                                                len(self.samples)))

    def get_partition_to_idxs(self):
        partition_to_idxs = {'train': [], 'test': []}
        prev_state = np.random.get_state()
        np.random.seed(123)

        idxs = np.arange(len(self.samples))
        n_test = int(self.test_frac * len(idxs))
        test_idxs = np.random.choice(idxs, replace=False, size=n_test).tolist()
        train_idxs = list(set(idxs) - set(test_idxs))

        partition_to_idxs['train'] = train_idxs
        partition_to_idxs['test'] = test_idxs

        np.random.set_state(prev_state)
        return partition_to_idxs
        
root = './../../data/'  
transform_train = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
transform_test = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
train_set = ImageWoof(train=True, transform=transform_train, root = root)
test_set = ImageWoof(train=False, transform=transform_test, root = root)
batch_size = 3000
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
data_iter1 = iter(train_loader)
X_sub1, _ = next(data_iter1)
X_sub1 = X_sub1.numpy()
print(X_sub1)
print(len(X_sub1))

=> done loading ImageWoof (train) with 7220 examples
=> done loading ImageWoof (test) with 1805 examples
[[[[ 1.2042983   1.3241715   1.5810429  ...  1.6495419   1.6495419
     1.6495419 ]
   [ 1.1871736   1.3241715   1.5981677  ...  1.6495419   1.6495419
     1.6495419 ]
   [ 1.1871736   1.3412963   1.6152923  ...  1.6495419   1.6495419
     1.6495419 ]
   ...
   [-0.01155927  0.03981499  0.09118938 ...  1.4954191   1.4954191
     1.4782944 ]
   [ 0.07406463  0.05693974  0.02269024 ...  1.42692     1.42692
     1.3926706 ]
   [ 0.15968838  0.07406463 -0.02868402 ...  1.3755459   1.3584211
     1.3241715 ]]

  [[ 1.2380953   1.3781513   1.6407562  ...  1.6582633   1.6582633
     1.6582633 ]
   [ 1.2205883   1.3606442   1.6407562  ...  1.6407562   1.6407562
     1.6407562 ]
   [ 1.2205883   1.3606442   1.6407562  ...  1.6057423   1.6057423
     1.6057423 ]
   ...
   [-0.12745096 -0.07492995 -0.02240895 ...  1.4306723   1.4306723
     1.4131653 ]
   [-0.03991595 -0.05742295 -0.09243695 .

In [6]:

concat_data = np.concatenate((X_sub, X_sub1), axis=0)
#print(concat_data)
print(len(concat_data))


6000


In [7]:

loaded_data = np.load('../../imagenette/2X_sub3000.npy')
print("原始数据长度:", len(loaded_data))
print("原始数据形状:", loaded_data.shape)

# 获取后1500个数据
last_1500_data = loaded_data[-1500:]
print("后1500个数据的长度:", len(last_1500_data))
print("后1500个数据的形状:", last_1500_data.shape)

原始数据长度: 3000
原始数据形状: (3000, 3, 224, 224)
后1500个数据的长度: 1500
后1500个数据的形状: (1500, 3, 224, 224)


In [8]:

concat_data = np.concatenate((concat_data, last_1500_data), axis=0)
#print(concat_data)
print(len(concat_data))


7500


In [9]:
# 创建全零标签数组
labels_sub = np.zeros(concat_data.shape[0])

# 将前一半样本的标签设置为0，后一半样本的标签设置为1
#labels_sub[concat_data.shape[0] // 2:] = 1
#labels_sub[:150] = 0
#labels_sub[150:] = 1
# 将concat_data分成三个部分，并为每个部分赋予相应的标签
labels_sub[:3000] = 0  # 前1500个元素的标签为0
labels_sub[3000:6000] = 1  # 接下来的1500个元素的标签为1
labels_sub[6000:] = 2  # 最后1500个元素的标签为2
#labels_sub[4500:] = 3

# 打印标签数组的长度和0形状
print(len(labels_sub))
print(labels_sub.shape)

7500
(7500,)


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# 转换为PyTorch的Tensor对象
X_tensor = torch.from_numpy(concat_data).float()
y_tensor = torch.from_numpy(labels_sub).long()


# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
print(X_train, X_test, y_train, y_test)
print("训练集长度:", len(X_train))
print("测试集长度:", len(X_test))
print("训练标签长度:", len(y_train))
print("测试标签长度:", len(y_test))
print(X_train.shape)
print(y_train.shape)

tensor([[[[-0.5424, -0.3198, -0.3541,  ..., -0.8507, -0.8678, -0.7137],
          [-0.5767, -0.3369, -0.3027,  ..., -0.8849, -0.9192, -0.8507],
          [-0.6452, -0.4226, -0.3541,  ..., -1.0048, -1.0562, -1.0390],
          ...,
          [ 0.6221,  0.5193,  0.4508,  ...,  0.2282,  0.1254,  0.2111],
          [ 0.6221,  0.5193,  0.3994,  ...,  0.1768,  0.0912,  0.1597],
          [ 0.6392,  0.5364,  0.3823,  ...,  0.1083,  0.0398,  0.0741]],

         [[-0.5826, -0.3550, -0.3901,  ..., -0.7927, -0.8102, -0.6527],
          [-0.6176, -0.3725, -0.3550,  ..., -0.8277, -0.8627, -0.7927],
          [-0.7052, -0.4776, -0.4076,  ..., -0.9503, -1.0028, -0.9853],
          ...,
          [ 0.4678,  0.3627,  0.2927,  ...,  0.0826, -0.0224,  0.0651],
          [ 0.4678,  0.3627,  0.2402,  ...,  0.0301, -0.0574,  0.0126],
          [ 0.4853,  0.3803,  0.2227,  ..., -0.0399, -0.1099, -0.0749]],

         [[-0.5147, -0.2881, -0.3230,  ..., -0.6890, -0.7064, -0.5495],
          [-0.5670, -0.3230, -

In [12]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

# 加载预训练的 ResNet-34 模型
model = models.resnet34(pretrained=True)

# 修改全连接层以进行3分类
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # 3分类

/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
# 定义逻辑回归模型
num_classes = 3
#model = MultiClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# 将数据和标签送入设备（如GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train = X_train.to(device)
y_train = y_train.to(device)
model = model.to(device)
# 创建训练数据加载器
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# 创建测试数据加载器
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

# 假设您的模型已经定义为 model
# 假设您的数据加载器已经定义为 train_loader
# 假设您的设备已经定义为 device

num_epochs = 55
model.train()

# 使用 Adam 优化器和 L2 正则化
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=1e-5)

# 交叉熵损失函数
criterion = nn.CrossEntropyLoss()

# 学习率调度器
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # 每100个批次打印一次
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.4f}")
            running_loss = 0.0

    # 更新学习率
    scheduler.step()

# 保存训练好的模型参数
torch.save(model.state_dict(), './image/attackimage1.pth')

print('训练完成')


[1, 100] loss: 0.7755
[2, 100] loss: 0.2921
[3, 100] loss: 0.1942
[4, 100] loss: 0.1537
[5, 100] loss: 0.1316
[6, 100] loss: 0.1069
[7, 100] loss: 0.1087
[8, 100] loss: 0.0921
[9, 100] loss: 0.0840
[10, 100] loss: 0.0734
[11, 100] loss: 0.0692
[12, 100] loss: 0.0593
[13, 100] loss: 0.0559
[14, 100] loss: 0.0520
[15, 100] loss: 0.0431
[16, 100] loss: 0.0464
[17, 100] loss: 0.0357
[18, 100] loss: 0.0374
[19, 100] loss: 0.0342
[20, 100] loss: 0.0305
[21, 100] loss: 0.0234
[22, 100] loss: 0.0306
[23, 100] loss: 0.0332
[24, 100] loss: 0.0281
[25, 100] loss: 0.0269
[26, 100] loss: 0.0302
[27, 100] loss: 0.0259
[28, 100] loss: 0.0318
[29, 100] loss: 0.0288
[30, 100] loss: 0.0263
[31, 100] loss: 0.0283
[32, 100] loss: 0.0304
[33, 100] loss: 0.0254
[34, 100] loss: 0.0259
[35, 100] loss: 0.0297
[36, 100] loss: 0.0238
[37, 100] loss: 0.0258
[38, 100] loss: 0.0258
[39, 100] loss: 0.0247
[40, 100] loss: 0.0240
[41, 100] loss: 0.0270
[42, 100] loss: 0.0363
[43, 100] loss: 0.0214
[44, 100] loss: 0.02

In [14]:
# 在测试集上评估模型性能
model.load_state_dict(torch.load('./image/attackimage1.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
test_loss = 0.0
correct = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
#       print(outputs)
        _, predicted = torch.max(outputs, 1)
#        print(predicted)
#        print(labels)
        if predicted == labels:
            correct += 1
    test_accuracy = correct / len(test_dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 0.9687


In [13]:
# 初始化计数器
TP = 0
FP = 0
TN = 0
FN = 0
model.load_state_dict(torch.load('./image/attackimage1.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
# 在测试集上评估模型性能
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        # 更新计数器
        for label, prediction in zip(labels, predicted):
            if label == 0 and prediction == 0:
                TP += 1
            elif label != 0 and prediction == 0:
                FP += 1
            elif label == 0 and prediction != 0:
                FN += 1
            elif label != 0 and prediction != 0:
                TN += 1

# 计算FPR和FNR
FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
FNR = FN / (FN + TP) if (FN + TP) > 0 else 0

print(f"FPR: {FPR:.4f}")
print(f"FNR: {FNR:.4f}")


FPR: 0.0207
FNR: 0.0381


In [16]:
TP = 0  # 真阳性
FN = 0  # 假阴性
model.load_state_dict(torch.load('./image/attackimage1.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        for label, prediction in zip(labels, predicted):
            if label == 1 and prediction == 1:
                TP += 1  # 真阳性：正确预测为类别1
            elif label == 1 and prediction != 1:
                FN += 1  # 假阴性：错误预测为非类别1

# 计算 FNR
FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
print(f"FNR for class 1: {FNR:.4f}")


FNR for class 1: 0.0377


In [18]:
import os.path as osp
import numpy as np
from torchvision.datasets import ImageFolder


class ImageNette(ImageFolder):
    test_frac = 0.2

    def __init__(self, train=True, transform=None, target_transform=None, root="./data/"):
        root = osp.join(root, 'imagenette-160')
        if not osp.exists(root):
            raise ValueError('Dataset not found at {}. Please download it from {}.'.format(
                root, 'https://s3.amazonaws.com/fast-ai-imageclas/imagenette-160.tgz'
            ))

        # Initialize ImageFolder
        super().__init__(root=osp.join(root, 'train'), transform=transform,
                         target_transform=target_transform)
        self.root = root

        self.partition_to_idxs = self.get_partition_to_idxs()
        self.pruned_idxs = self.partition_to_idxs['train' if train else 'test']

        # Prune (self.imgs, self.samples to only include examples from the required train/test partition
        self.samples = [self.samples[i] for i in self.pruned_idxs]
        self.imgs = self.samples

        print('=> done loading {} ({}) with {} examples'.format(self.__class__.__name__, 'train' if train else 'test',
                                                                len(self.samples)))

    def get_partition_to_idxs(self):
        partition_to_idxs = {
            'train': [],
            'test': []
        }

        # Note: we perform a 80-20 split of imagenet training
        # While this is not necessary, this is to simply to keep it consistent with the paper
        prev_state = np.random.get_state()
        np.random.seed(123)

        idxs = np.arange(len(self.samples))
        n_test = int(self.test_frac * len(idxs))
        test_idxs = np.random.choice(idxs, replace=False, size=n_test).tolist()
        train_idxs = list(set(idxs) - set(test_idxs))

        partition_to_idxs['train'] = train_idxs
        partition_to_idxs['test'] = test_idxs

        np.random.set_state(prev_state)

        return partition_to_idxs
        
root = './../../data/'  # 数据集存储的根目录]
transform_train = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
transform_test = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
#train_set = ImageNette(train=True, transform=transform_train, root = root)
#test_set = ImageNette(train=False, transform=transform_test, root = root)
train_set = ImageWoof(train=True, transform=transform_train, root = root)
test_set = ImageWoof(train=False, transform=transform_test, root = root)
batch_size = 1 # 每个批次的样本数
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

IN_count = 0
JBDA_count = 0
Knockoff_count = 0
input_count = 0

import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

# 加载预训练的 ResNet-34 模型
model = models.resnet34(pretrained=True)

# 修改全连接层以进行3分类
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # 3分类
model.load_state_dict(torch.load('./image/attackimage1.pth'))
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# 在测试集上评估模型性能
model.eval()
with torch.no_grad():
    for inputs, _ in tqdm(train_loader, desc="Validation Progress"):
        input_count += 1
        inputs = inputs.to(device)
        outputs = model(inputs)
#        print(outputs)
        _, predicted = torch.max(outputs, 1)
        if predicted == 0 :
            IN_count += 1
        elif predicted == 1 :
            Knockoff_count += 1
        elif predicted == 2:
            JBDA_count += 1
    print(f"input: {input_count:}")
    print(f"IN: {IN_count:}")
    print(f"Knockoff Data: {Knockoff_count:}")
    print(f"JBDA Data: {JBDA_count:}")

=> done loading ImageWoof (train) with 7220 examples
=> done loading ImageWoof (test) with 1805 examples


Validation Progress: 100%|██████████████████████████████████████| 7220/7220 [01:04<00:00, 111.47it/s]

input: 7220
IN: 249
Knockoff Data: 6971
JBDA Data: 0
